# Codigos usados

In [2]:
import nltk
import json
import random
import numpy as np
import re
import unicodedata
from nltk.stem import PorterStemmer
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau

In [355]:
# Cargar el archivo JSON
with open('Biblioteca_de_intenciones.json', 'r', encoding='utf-8') as file:
    intents = json.load(file)

In [356]:
# Inicializar el stemmer y el encoder
stemmer = PorterStemmer()
encoder = LabelEncoder() # Similar al Integer Encoding

In [357]:
# Preparar los datos
patterns = []  # Base de conocimientos por cada intención
responses = [] # Respuesta de la intención
tags = []      # Intents

# Llenar las listas con los datos del JSON
for intent in intents['intents']:
    for pattern in intent['patterns']:
        patterns.append(pattern)
        responses.append(intent['responses'])
        tags.append(intent['tag']) # Intent

In [ ]:
# Tokenización y Stemming
def tokenize_and_stem(sentence):
    words = nltk.word_tokenize(sentence)
    words = [stemmer.stem(w.lower()) for w in words if w.isalnum()]
    return words

# Normalización del texto y implementacion de los stopwords en español
def word_normalization(words):
    texto_normalizado = unicodedata.normalize('NFD', words)

    # Eliminar caracteres especiales y acentos
    texto_sin_acentos = re.sub(r'[\u0300-\u036f]', '', texto_normalizado)

    # Eliminar signos especiales
    texto_limpio = re.sub(r'[^a-zA-Z0-9\s]', '', texto_sin_acentos).split() # Se genera una LISTA de palabras en minúscula
    
    stops = set(stopwords.words("spanish")) # Eliminamos palabras que no afecten el significado de la oración
    
    meaningful_words = [w for w in texto_limpio if not w in stops] # Guardamos las stopwords
    return(" ".join( meaningful_words ))

In [360]:
# Crear el Bag of Words
all_words = []
for pattern in patterns:
    words = word_normalization(pattern)
    words = tokenize_and_stem(pattern)
    all_words.extend(words)

all_words = sorted(list(set(all_words)))

# Crear el Bag of Words (BoW)
vectorizer = CountVectorizer(vocabulary=all_words)
X_train = vectorizer.transform(patterns).toarray()

# Codificar las etiquetas
y_train = encoder.fit_transform(tags)

# Entrenamiento con redes densas

In [363]:
# Crear la red neuronal
model = Sequential()
model.add(Dense(128, input_shape=(X_train.shape[1],), activation='relu', kernel_regularizer='l1l2'))
model.add(Dense(64, activation='relu', kernel_regularizer='l1l2'))
model.add(Dense(32, activation='relu', kernel_regularizer='l1l2'))
model.add(Dense(len(set(tags)), activation='softmax'))

# Compilar el modelo
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=200, batch_size=50, verbose=1)

Epoch 1/200


c:\Users\ortiz\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - accuracy: 0.0298 - loss: 3.5808 
Epoch 2/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0627 - loss: 3.5533 
Epoch 3/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0555 - loss: 3.5229 
Epoch 4/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.1003 - loss: 3.4824 
Epoch 5/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.1398 - loss: 3.4251 
Epoch 6/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.1926 - loss: 3.3514 
Epoch 7/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.2036 - loss: 3.2471 
Epoch 8/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.2210 - loss: 3.1387 
Epoch 9/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.2333 - loss: 2.9600 
Epoch 10/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2456 - loss: 2.7581 
Epoch 11/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.3260 - loss: 2.6145 
Epoch 12/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.3937 - loss: 2.4946 


# Funciones para clasificar los tipos de sangre segun la entidad detectada (tipo de sangre A+, O+, AB-, etc.).

In [ ]:
import joblib
import pandas as pd

# Importamos los modelos cargados
clasificar_recepcion = joblib.load('modelo_recepcion.pkl') # Carga del modelo de recepcion de sangre.
clasificar_donacion = joblib.load('modelo_donacion.pkl') # Carga del modelo de donacion de sangre.

# Codigo de clasificacion para el asistente virtual
def clasificador_donacion(datos):

    # Preparar los datos
    X_datos = pd.DataFrame(datos, columns=['O-', 'O+', 'B-', 'B+', 'A-', 'A+', 'AB-', 'AB+'])

    # Predecir la etiqueta de donación
    prediccion_datos = clasificar_donacion.predict(X_datos)
    
    return prediccion_datos

def clasificador_recepcion(datos):

    # Preparar los datos
    X_datos = pd.DataFrame(datos, columns=['O-', 'O+', 'B-', 'B+', 'A-', 'A+', 'AB-', 'AB+'])
    
    # Predecir la etiqueta de recepcion
    prediccion_datos = clasificar_recepcion.predict(X_datos)
    
    return prediccion_datos


# transformar tipo de sangre a hotencoded desde asistente virtual
def hotencode_sangre(sangre):
    sangres = ['O-', 'O+', 'B-', 'B+', 'A-', 'A+', 'AB-', 'AB+']
    return [1 if s == sangre else 0 for s in sangres]

# Reconstruimos el tipo de sangre a partir de hotencode
def decodificar_sangre(hotencoded):
    sangres = ['O-', 'O+', 'B-', 'B+', 'A-', 'A+', 'AB-', 'AB+']
    return [sangres[i] for i in range(8) if hotencoded[i] == 1]

# entrada de sangre del asistente virtual
def recepcion(type_blood):
    # transformar el tipo de sangre a hotencoded
    hotencoded = hotencode_sangre(type_blood.upper())
    
    # predecir la etiqueta de recepcion
    prediccion = clasificador_recepcion([hotencoded])[0]

    # decodificamos la prediccion
    sangre = decodificar_sangre(prediccion)
    return sangre

def donacion(type_blood):
    # transformar el tipo de sangre a hotencoded
    hotencoded = hotencode_sangre(type_blood.upper())
    
    # predecir la etiqueta de donacion
    prediccion = clasificador_donacion([hotencoded])[0]
    
    # decodificamos la prediccion
    sangre = decodificar_sangre(prediccion)
    return sangre

# Funcion para dar la respuesta mas probable y si detecta el tipo de sangre ejecutar el clasificador

In [4]:
# Función para obtener la respuesta más probable
def chatbot_response(text):
    # Preprocesar el texto de entrada
    text_words = word_normalization(text)
    text_words = tokenize_and_stem(text)
    bow = np.zeros(len(all_words))
    
    for word in text_words:
        if word in all_words:
            bow[all_words.index(word)] = 1
    
    # Predecir la categoría
    prediction = model.predict(np.array([bow]))[0]
    tag = encoder.inverse_transform([np.argmax(prediction)])

    # Buscar una respuesta dentro de la categoría
    for intent in intents['intents']:
        if intent['tag'] == tag:

            # Detectar la entidad
            tipos_sangre_posibles = ["O+", "O-", "A+", "A-", "B+", "B-", "AB+", "AB-"]
            palabras = text.split() # Dividir el texto en palabras
            coincidencias = []  # Lista para almacenar los tipos encontrados

            for palabra in palabras:
                
                # Eliminar puntuación de cada palabra
                palabra_limpia = palabra.strip(",.?!")

                # Comparar con los tipos de sangre
                if palabra_limpia in tipos_sangre_posibles:
                    coincidencias.append(palabra_limpia)

            if coincidencias:
                entity = coincidencias[0]  # Capturar la entidad
                
                # Generar respuesta dinámica
                modelo_recepcion = recepcion(entity)  # Obtener tipos compatibles
                modelo_donacion = donacion(entity)  # (Función similar para donación)
                
                # Formatear la respuesta
                respuesta = random.choice(intent['responses']).format(
                    entity=entity,
                    modelo_recepcion=", ".join(modelo_recepcion),
                    modelo_donacion=", ".join(modelo_donacion)
                )
                return respuesta
            return random.choice(intent['responses'])

In [ ]:
# Probar el chatbot
print("¡Hola! Soy tu chatbot. Escribe 'salir' para terminar.")
while True:
    user_input = input("Tú: ")
    if user_input.lower() == "salir":
        break
    response = chatbot_response(user_input)
    print(f"Chatbot: {response}")